## <span style='color:purple '> 1. Import Libraries </span>

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#set floating point precision option for pandas
pd.set_option('display.float_format', lambda x: '%.4f' % x)

import seaborn as sns
sns.set_context('paper', font_scale=1.3)
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')

from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller, pacf

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error , mean_absolute_error
%matplotlib inline

import math

#Import necessary functions from keras
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import *
from keras.callbacks import EarlyStopping

#to read zipfile
import zipfile

## <span style ='color:purple'> 2. Import Data </span>

In [9]:
zip_file_path = 'cosumption_data.zip'
file_name = 'household_power_consumption.txt'

with zipfile.ZipFile(zip_file_path,'r') as folder:
    file_list = folder.namelist()
    for file in file_list:
        if file == file_name:
            with folder.open(file_name) as csv_file:
                df = pd.read_csv(csv_file, delimiter=';')
    

In [10]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0000
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0000
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0000
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0000
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0000


In [ ]:
df.